In [ ]:
%pip install transformers datasets torch

In [ ]:
import torch
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('MonoHime/ru_sentiment_dataset')

In [ ]:
train_data = dataset['train']
validation_data = dataset['validation']

In [ ]:
#from transformers import AutoTokenizer, AutoModel
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
#model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = BertModel.from_pretrained("cointegrated/rubert-tiny2")

In [ ]:
print(tokenizer.vocab_size)

83828


In [ ]:
#print(model.bert.embeddings)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEUTRAL", 1: "POSITIVE", 2: "NEGATIVE"}
label2id = {"NEUTRAL": 0, "POSITIVE": 1, "NEGATIVE" :2}

In [ ]:
! pip install -U accelerate

In [ ]:
from transformers import AutoModel, TrainingArguments, Trainer

model = AutoModel.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=3, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

training_args = TrainingArguments(
    output_dir="ru-sentiment",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,

)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`